# Nondegenerate internal squeezing
### Generation of Figures 3--6 for the paper
James Gardner, December 2021

In [ ]:
# imports interferometer class. also loads dependencies, e.g. numpy 
from interferometer import *

%config InlineBackend.figure_format = 'svg'
plt.rcParams.update({'font.size': 18})

Fig 3 - noise, signal, and sensitivity versus frequency 

In [ ]:
# fig 3 for paper, nIS, vertical NSNSR
# NEED TO ANNOTATE WITH INKSCAPE
# see workshop N_S_NSR_compact for the VBA-preferred representation
liIFO = IFO(2e-6, 4e3, None, 3e6, None, 0.046, 200, 2*pi*5000, 2*pi*500, 0)

paramsList = [
    [0,    *losses0,*signalRO],
    [0.80, *losses0,*signalRO],
    [0.95, *losses0,*signalRO],
    [0.986,*losses0,*signalRO],
    [0.986,*losses0,*signalRO]]
extSqzFactor_List = (1, 1, 1, 1, 1/10)
freq_tuple = (2, 5e4, 300) #300 points
f_List = np.logspace(np.log10(freq_tuple[0]),np.log10(freq_tuple[1]),num=freq_tuple[2])

color_List=((0,0,1,1),(1,0,0,0.4),(1,0,0,0.8),(1,0,0,1),(1,0,1,1))
fmt_List=('--','','','','-.')
save_path='nIS_N_S_NSR_vertical.pdf'

plt.rcParams.update({'font.size': 12})
fig, axs = plt.subplots(3, 1, sharex=True, figsize=(9/cm_to_inch, 20/cm_to_inch),
                        gridspec_kw={'height_ratios': [2, 2, 3]})
plt.subplots_adjust(hspace=0.05)

num_curves = len(paramsList)

radiation_pressure_List = np.full(num_curves, True)
wm_List = np.full(num_curves, 0)
psi3_List = np.full(num_curves, 0)

axs[0].axhline(0, color='grey', linewidth=1)

for i, params in enumerate(paramsList[:-1]):  
    axs[0].plot(f_List, 20*np.log10(np.array(liIFO.noiseList_vs_freq(params, freq_tuple, radiation_pressure_on=radiation_pressure_List[i], extSqzFactor=extSqzFactor_List[i], wm=wm_List[i], psi3=psi3_List[i]))), fmt_List[i], color=color_List[i])
    axs[1].loglog(f_List, liIFO.signalList_vs_freq(params, freq_tuple, radiation_pressure_on=radiation_pressure_List[i], extSqzFactor=extSqzFactor_List[i], wm=wm_List[i], psi3=psi3_List[i]), fmt_List[i], color=color_List[i])
    axs[2].loglog(f_List, liIFO.sensList_vs_freq(params, freq_tuple, radiation_pressure_on=radiation_pressure_List[i], extSqzFactor=extSqzFactor_List[i], wm=wm_List[i], psi3=psi3_List[i]), fmt_List[i], color=color_List[i])

# add external squeezing to just to sensitivity plot
axs[2].loglog(f_List, liIFO.sensList_vs_freq(paramsList[-1], freq_tuple, radiation_pressure_on=radiation_pressure_List[-1], extSqzFactor=extSqzFactor_List[-1], wm=wm_List[-1], psi3=psi3_List[-1]), fmt_List[-1], color=color_List[-1])
    
axs[2].set_ylim((None, 1e-21))
axs[0].set_ylim((None, 40))
axs[2].set_xlim(freq_tuple[0], freq_tuple[1])
axs[2].set_xlabel('frequency (Hz)')
axs[0].set_ylabel('quantum noise (dB)')
axs[1].set_ylabel('signal response')
axs[2].set_ylabel('sensitivity ($\mathrm{Hz}^{-1/2}$)')

from matplotlib.lines import Line2D
line1 = Line2D([0], [0], label='no squeezing', color='b', linestyle='--')
line2 = Line2D([0], [0], label='nondegenerate\ninternal squeezing', color='r')
axs[2].legend(handles=[line1, line2], handlelength=1, frameon=False, borderpad=0,
              loc='upper center', bbox_to_anchor=(0.52, 1))

fig.savefig(save_path, bbox_inches = "tight")        
plt.close(fig) 

Fig 4 - tolerance to detection loss

In [ ]:
# combined tolerance plot
liIFO = IFO(2e-6, 4e3, None, 3e6, None, 0.046, 200, 2*pi*5000, 2*pi*500, 0)
intra_losses= losses0[:-1]
intra_losses1 = intra_losses

# "boring plot" tolerance to detection loss: no sqz., nondeg. int. sqz.
# looks different to s11 because of xRatio = 98.6% instead of 95%
Rpd_List1 = (0, 0.2, 0.4, 0.6)
freq_tuple1 = (2, 5e4, 300)
f_List = np.logspace(np.log10(freq_tuple1[0]), np.log10(freq_tuple1[1]), num=freq_tuple1[2])

# changing threshold ratio from 0.986 to 0.95 for consistency in paper
xRatioTol = 0.95
paramsList = [
    *([0, *intra_losses1, Rpd, *signalRO] for Rpd in Rpd_List1),
    *([xRatioTol, *intra_losses1, Rpd, *signalRO] for Rpd in Rpd_List1)]
labels1 = [*('' for _ in Rpd_List1),
           *('{:.0f}%'.format(Rpd*100) for Rpd in Rpd_List1),
            'detection loss']
linestyles1 = ['--','--','--','--','-','-','-','-']
colors1 = [(0,0,1,1),(0,0,1,0.75),(0,0,1,0.5),(0,0,1,0.25),
           (1,0,0,1),(1,0,0,0.75),(1,0,0,0.5),(1,0,0,0.25)]

sensListList = np.empty((len(paramsList), len(f_List)))
for i, params in enumerate(paramsList):
    sensListList[i] = liIFO.sensList_vs_freq([*params], freq_tuple1)

from scipy.optimize import minimize

def peak_sens_given_Rpd2(Rpd, x0=np.random.uniform(100, 1000, 1),
                         xRatio=xRatio0, extSqzFactor1=1, intra_losses1=losses0[:-1]):
    args0 = (xRatio, *intra_losses1, Rpd, *signalRO)
    bounds0 = ((1e2, 1e4),)
    
    def log_sens_given_f_singleton(f_arr, *args):
        return np.log10(liIFO.ASDSh(f_arr[0], *args, extSqzFactor=extSqzFactor1))
    
    return minimize(log_sens_given_f_singleton, x0, args=args0, bounds=bounds0, options={'maxiter': 10})    
    
lossless_peak_freq0 = peak_sens_given_Rpd2(0, x0=5e3, xRatio=0, extSqzFactor1=1, intra_losses1=intra_losses1).x[0]    
result1 = peak_sens_given_Rpd2(0, x0=2e3, xRatio=xRatioTol, extSqzFactor1=1, intra_losses1=intra_losses1)
lossless_peak_freq1 = result1.x[0]

# probe sensitivity (at lossless peak frequency) in dB versus detection loss
num_samples = 50 # 300 points required for old scatter technique
max_Rpd = 0.8 # odd that this isn't max(Rpd_List1)
Rpd_List = np.linspace(0, max_Rpd, num=num_samples)

def dBsensList_probe_vs_Rpd(lossless_peak_freq, xRatio1, extSqzFactor1): 
    global _sens_for_Rpd
    def _sens_for_Rpd(Rpd):
        return liIFO.ASDSh(lossless_peak_freq, xRatio1, *intra_losses1, Rpd, *signalRO, extSqzFactor=extSqzFactor1)

    return 20*np.log10(p_map(_sens_for_Rpd, Rpd_List)/_sens_for_Rpd(0))

dBsensListList = np.empty((4, num_samples))
# no sqz
dBsensListList[0] = dBsensList_probe_vs_Rpd(lossless_peak_freq0, 0, 1)
# ext sqz
dBsensListList[1] = dBsensList_probe_vs_Rpd(lossless_peak_freq0, 0, 1/10)
# nIS
dBsensListList[2] = dBsensList_probe_vs_Rpd(lossless_peak_freq1, xRatioTol, 1)
# nIS + ext sqz
dBsensListList[3] = dBsensList_probe_vs_Rpd(lossless_peak_freq1, xRatioTol, 1/10)

dataset = np.empty((num_samples, 5))
dataset[:,0], dataset[:,1:] = Rpd_List, dBsensListList.transpose()
np.save('data_of_tolerance_to_detection_loss/tolerance_(Rpd,conv,ext,nIS,ext+nIS).npy', dataset)

In [ ]:
# fig 4 for paper, as above but vertical
plt.rcParams.update({'font.size': 12})
cm_to_inch = 2.54
# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9/cm_to_inch, 14/cm_to_inch))

fig = plt.figure(figsize=(9/cm_to_inch, 16/cm_to_inch))
gs = fig.add_gridspec(2, 1, hspace=0.25, height_ratios=[3, 2])
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])

for i, sensList in enumerate(sensListList):
    ax1.loglog(f_List, sensList, label=labels1[i], linestyle=linestyles1[i], color=colors1[i])

ax1.text(0.89, 0.03, '(a)', fontsize=14, transform=ax1.transAxes)
ax1.legend(title=labels1[-1], handlelength=1, ncol=2, columnspacing=0.5, frameon=False, borderpad=0,)
ax1.set_xlabel('frequency (Hz)')
ax1.set_ylabel('sensitivity ($\mathrm{Hz}^{-1/2}$)')
ax1.set_xlim((freq_tuple1[0], freq_tuple1[1]))
ax1.set_xticks([1e1, 1e2, 1e3, 1e4])
ax1.set_xticklabels(['10', '100', '1000', '$10^4$'])
ax1.set_ylim((None, 1e-21))

hline_width = 1.8
ax1.hlines(sensListList[0].min(),
           f_List[sensListList[0].argmin()]/hline_width,
           f_List[sensListList[0].argmin()]*hline_width, colors='k') #log(peak*2^(+-1)) = log(peak) +- log(2)
ax1.hlines(sensListList[4].min(),
           f_List[sensListList[4].argmin()]/hline_width,
           f_List[sensListList[4].argmin()]*hline_width, colors='k')

from matplotlib import cm
from matplotlib.colors import ListedColormap

N = 256
cmp_blue = ListedColormap(np.column_stack((np.linspace(0, 1, N), np.linspace(0, 1, N), np.ones(N), np.ones(N))))
cmp_red = ListedColormap(np.column_stack((np.ones(N), np.linspace(0, 1, N), np.linspace(0, 1, N), np.ones(N))))

# if colouring along the line is used, need to add the ability to dash the curve
# plot_coloured_line_segments(Rpd_List, dBsensListList[0], np.abs(Rpd_List/np.max(Rpd_List)), cmp_blue, ax=ax2)
# plot_coloured_line_segments(Rpd_List, dBsensListList[2], np.abs(Rpd_List/np.max(Rpd_List)), cmp_red, ax=ax2)
ax2.plot(Rpd_List, dBsensListList[0], linestyle='--', color='b')
ax2.plot(Rpd_List, dBsensListList[2], linestyle='-', color='r')

ax2.text(0.89, 0.04, '(b)', fontsize=14, transform=ax2.transAxes)
ax2.set_xlabel('detection loss ($\%$)')
ax2.set_ylabel('reduction in peak\nsensitivity (dB)')
ax2.set_xticks(np.arange(0, max_Rpd+0.2, step=0.2))
ax2.set_xticklabels(('{:.0f}'.format(i) for i in np.arange(0, 100*(max_Rpd+0.2), step=20)))
ax2.set_xlim(0, max_Rpd)
ax2.set_ylim(0, 7.5)

import matplotlib.lines as mlines
blue_marker = mlines.Line2D([], [], color='b', label='no squeezing', linestyle='--')
red_marker = mlines.Line2D([], [], color='r', label='nondegenerate\ninternal squeezing')
ax2.legend(handles=[blue_marker, red_marker], handlelength=1, frameon=False, borderpad=0)

# fig.align_xlabels()
# plt.show()
fig.savefig('fig4_nIS_tolerance_to_detection_loss_sens_reduction_combined_vertical.pdf', bbox_inches='tight')
plt.close(fig)

Fig 5 - idler readout (fixed vs variational)

In [ ]:
ifo = IFO(2e-6, 4e3, None, 3e6, None, 0.046, 200, 2*pi*5000, 2*pi*500, 2*pi*5)
freq_tuple1 = (30, 5e3, 100)

In [ ]:
# generate and save data for variational idler
# DO NOT RUN if just plotting and data already saved
ifo.save_idler_varRO(freq_tuple1, losses0, 'realistic_losses')
ifo.save_idler_varRO(freq_tuple1, (0,0,0,0), 'lossless')

In [ ]:
fixed_signalRO_list = ifo.sensList_vs_freq([xRatio0, *losses0, *signalRO], freq_tuple1)
fixed_signalRO_list_lossless = ifo.sensList_vs_freq([xRatio0, *(0,0,0,0), *signalRO], freq_tuple1)

# fig 5 : fixed signal vs fixed idler vs variational idler RO
plt.rcParams.update({'font.size': 12})
cm_to_inch = 2.54
fig, ax = plt.subplots(figsize=(9/cm_to_inch,6/cm_to_inch)) # columnwidth~=8.6 cm

# load and plot variational idler RO
var_idler_lossy =     np.load('./optimal_angles/data_idler_(freq,psi1,varSens,fixedSens)--realistic_losses.npy')
var_idler_lossless =  np.load('./optimal_angles/data_idler_(freq,psi1,varSens,fixedSens)--lossless.npy')
ax.loglog(var_idler_lossy[:,0], var_idler_lossy[:,3], linestyle=(0, (5, 2, 1, 2)), color='sienna', label='fixed idler')
ax.loglog(var_idler_lossy[:,0], var_idler_lossy[:,2], 'r', label='variational idler')
ax.loglog(var_idler_lossless[:,0], var_idler_lossless[:,2], color=(1,0,0,0.5), label='lossless variational\nidler')

ax.loglog(var_idler_lossy[:,0], fixed_signalRO_list, color=(64/255,224/255,208/255,1), linestyle='--', label='fixed signal')
ax.loglog(var_idler_lossy[:,0], fixed_signalRO_list_lossless, color=(64/255,224/225,208/255,0.5), linestyle='--', label='lossless fixed\nsignal')
sql_list = ifo.sql_list_vs_freq(freq_tuple1)
ax.loglog(var_idler_lossy[:,0], sql_list, ls='dotted', color='grey', label='SQL')

ax.set_ylim((3e-25, 1e-23))
ax.set_xlim(*freq_tuple1[:-1])
ax.set_xlabel('frequency (Hz)')
ax.set_ylabel('sensitivity ($\mathrm{Hz}^{-1/2}$)')

# frameon=False, borderpad=0,
handles, labels = ax.get_legend_handles_labels()
# leg_order = lambda x : [x[1], x[0], *x[2:]]
import matplotlib.lines as mlines
explicit_handle = mlines.Line2D([], [], color='sienna', linestyle=(0, (3, 1, 1, 1)))
ax.legend((explicit_handle, *handles[1:]), labels, fontsize=12, ncol=2,
          columnspacing=0.5, loc='lower center', bbox_to_anchor=(0.35, -0.9), handlelength=1.1)

# fig.tight_layout()
# plt.show()
fig.savefig('fig5_idlerRO_fixed_vs_variational.pdf', bbox_inches='tight')
plt.close(fig)

Fig 6 - optimal filter (with and without variational readout)

In [ ]:
ifo = IFO(2e-6, 4e3, None, 3e6, None, 0.046, 200, 2*pi*5000, 2*pi*500, 2*pi*5)
freq_tuple1 = (30, 5e3, 100)

In [ ]:
# generate and save data for variational idler, variational signal, optimal w and w/o variational readout
# DO NOT RUN if just plotting and data already saved
ifo.save_idler_varRO(freq_tuple1, losses0, 'realistic_losses')
ifo.save_idler_varRO(freq_tuple1, (0,0,0,0), 'lossless')

ifo.save_signal_varRO(freq_tuple1, losses0, 'realistic_losses')
ifo.save_signal_varRO(freq_tuple1, (0,0,0,0), 'lossless')

# copying over data from old script in workshop, to-do: check that these functions return the same data
ifo.save_optimal_filter(freq_tuple1, losses0, 'realistic_losses')
ifo.save_optimal_filter(freq_tuple1, (0,0,0,0), 'lossless')

ifo.save_optimal_filter_no_variational(freq_tuple1, losses0, 'realistic_losses')
ifo.save_optimal_filter_no_variational(freq_tuple1, (0,0,0,0), 'lossless')

In [ ]:
# fig 6 as above but vertically stacked
freq_tuple1 = (30, 5e3, 100)
plt.rcParams.update({'font.size': 12})
cm_to_inch = 2.54
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(9/cm_to_inch,12/cm_to_inch))
plt.subplots_adjust(hspace=0.05)

# variational signal RO
var_signal_lossy =     np.load('./optimal_angles/data_signal_(freq,psi0,varSens,fixedSens)--realistic_losses.npy')
var_signal_lossless =  np.load('./optimal_angles/data_signal_(freq,psi0,varSens,fixedSens)--lossless.npy')
ax1.loglog(var_signal_lossy[:,0], var_signal_lossy[:,2], '--', color='turquoise', label='variational signal')
ax2.loglog(var_signal_lossless[:,0], var_signal_lossless[:,2], '--',
          color='turquoise', label='lossless variational signal')

var_idler_lossy =     np.load('./optimal_angles/data_idler_(freq,psi1,varSens,fixedSens)--realistic_losses.npy')
var_idler_lossless =  np.load('./optimal_angles/data_idler_(freq,psi1,varSens,fixedSens)--lossless.npy')
# fixed idler
ax1.loglog(var_idler_lossy[:,0], var_idler_lossy[:,3], linestyle=(0, (5, 2, 1, 2)), color='sienna', label='fixed idler', zorder=2)
ax2.loglog(var_idler_lossless[:,0], var_idler_lossless[:,3], linestyle=(0, (5, 2, 1, 2)), color='sienna', label='lossless fixed idler', zorder=2)
# variational idler RO
ax1.loglog(var_idler_lossy[:,0], var_idler_lossy[:,2], 'r', label='variational idler')
ax2.loglog(var_idler_lossless[:,0], var_idler_lossless[:,2], 'r', label='lossless variational idler')

# gentle shading
shade_color = 'whitesmoke'
ax1.fill_between(var_signal_lossy[:,0], np.minimum(var_signal_lossy[:,2], var_idler_lossy[:,2]), 1e-23, facecolor=shade_color, interpolate=True)
ax2.fill_between(var_signal_lossless[:,0], np.minimum(var_signal_lossless[:,2], var_idler_lossless[:,2]), 1e-23, facecolor=shade_color, interpolate=True)

# optimal filter with no variational readout
opt_no_var_lossy =    np.load('./optimal_angles/data_optimal_no_var_(freq,psi2,psi3,sens)--realistic_losses.npy')
opt_no_var_lossless = np.load('./optimal_angles/data_optimal_no_var_(freq,psi2,psi3,sens)--lossless.npy')
# dashed linestyle: (0, (5, 5))
ax1.loglog(opt_no_var_lossy[:,0], opt_no_var_lossy[:,-1], color=(1,0,1,0.5), linestyle='-', label='filter only', zorder=1)
ax2.loglog(opt_no_var_lossless[:,0], opt_no_var_lossless[:,-1], color=(1,0,1,0.5), linestyle='-', label='optimal, no variational', zorder=1)

# optimal filter
complex_lossy =     np.load('./optimal_angles/data_freq_angles_complex_sens--realistic_losses.npy')
complex_lossless =  np.load('./optimal_angles/data_freq_angles_complex_sens--lossless.npy')
# dashed linestyle offsets: (-3.3, (5, 5)); (.7, (5, 5))
ax1.loglog(complex_lossy[:,0], complex_lossy[:,-1], 'darkviolet', linestyle=(0, (5, 2, 1, 2, 1, 2)), label='filter with\nvariation')
ax2.loglog(complex_lossless[:,0], complex_lossless[:,-1], 'darkviolet', linestyle=(0, (5, 2, 1, 2, 1, 2)), label='lossless optimal filter')

sql_list = ifo.sql_list_vs_freq((1, 5e3, 100))
ax1.loglog(np.logspace(np.log10(1), np.log10(5e3), 100), sql_list, ls='dotted', color='grey', label='SQL')
ax2.loglog(np.logspace(np.log10(1), np.log10(5e3), 100), sql_list, ls='dotted', color='grey', label='SQL')

ax1.set_ylim((1.5e-25, 1e-23))
ax2.set_ylim((1.5e-25, 1e-23))
ax1.set_ylabel('sensitivity ($\mathrm{Hz}^{-1/2}$)')
ax2.set_ylabel('sensitivity ($\mathrm{Hz}^{-1/2}$)')
ax2.set_xlim(*freq_tuple1[:-1])
ax2.set_xlabel('frequency (Hz)')

ax1.set_title('(a) lossy',    fontsize=12, y=1.0, pad=-12, style='italic')
ax2.set_title('(b) lossless', fontsize=12, y=1.0, pad=-12, style='italic')
handles, labels = ax1.get_legend_handles_labels()

box = ax2.get_position()
# ax2.legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, borderpad=0)
leg_order = lambda x : [x[1], x[2], x[0], x[-1], x[3], x[4]]

import matplotlib.lines as mlines
orange_handle = mlines.Line2D([], [], color='sienna', linestyle=(0, (3, 1, 1, 1)))
# magenta_handle = mlines.Line2D([], [], color='m', linestyle='--')
blue_handle = mlines.Line2D([], [], color='darkviolet', linestyle=(0, (2, 1, 1, 1, 1, 1)))
ax2.legend((orange_handle, handles[2], handles[0], handles[-1], handles[3], blue_handle), leg_order(labels),
           loc='lower center', bbox_to_anchor=(0.35, -0.91),
          ncol=2, columnspacing=0.5, fontsize=12, handlelength=1.1)

# plt.show()
fig.savefig('fig6_optimal_filter_vs_variational_readouts_vs_optNoVar_vertical.pdf', bbox_inches='tight')
plt.close(fig)